# Deploy the model to a sagemaker endpoint

In [2]:
import os
import boto3
import sagemaker
from sagemaker.pytorch.model import PyTorchModel, PyTorchPredictor

In [ ]:
# modify these variables according to your configuration
bucket = 's3-bucket-name'
model_artifacts = 'model_artifacts.tar.gz'
model_s3_uri = os.path.join('s3://', bucket, model_artifacts)
endpoint_name = 'resnet18-sagemaker-endpoint'
role = 'AmazonSageMaker-ExecutionRole-...'

In [ ]:
# archive and zip codes folder and upload it to s3
!tar -cvzf {model_artifacts} inference_code/
!aws s3 cp {model_artifacts} {model_s3_uri}

In [ ]:
sess = sagemaker.Session()

# instance types: https://aws.amazon.com/sagemaker/pricing/?p=pm&c=sm&z=4
instance_type = 'ml.t2.medium'

# Deploy to an endpoint with python 3.8 and pytorch 1.10.2
model = PyTorchModel(
    model_data=model_s3_uri,
    source_dir='inference_code',
    entry_point='inference.py',   
    role=role,
    sagemaker_session=sess,
    framework_version='1.10.2',
    py_version='py38'
)

client = boto3.client('sagemaker')
# Delete model in order to force the script to create it again
try:
    client.delete_model(ModelName=endpoint_name)    
except Exception as e:
    # model does not exist
    pass   

# Delete endpoint configuration in order to force the script to create it again
try:
    client.delete_endpoint_config(EndpointConfigName=endpoint_name)
except Exception as e:
    # endpoint configuration does not exist
    pass 

try:
    # create endpoint
    predictor = model.deploy(
        instance_type=instance_type,
        initial_instance_count=1,
        endpoint_name=endpoint_name
    )
except Exception as e:
    # endpoint already exists, so update it
    predictor = PyTorchPredictor(endpoint_name=endpoint_name)
    predictor.update_endpoint(
        model_name=model.name,
        instance_type=instance_type,
        initial_instance_count=1,
    )

# Sagemaker inference

In [4]:
from PIL import Image

image_path = 'data/dog.jpg'
image = Image.open(image_path)

In [6]:
from sagemaker.pytorch.model import PyTorchPredictor

predictor = PyTorchPredictor(endpoint_name=endpoint_name)
probabilities = predictor.predict(image)

In [ ]:
# Download imagenet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
!mv imagenet_classes.txt data/imagenet_classes.txt

In [8]:
# Read the labels
with open("data/imagenet_classes.txt", "r") as f:
    labels = [s.strip() for s in f.readlines()]

# Predicted class
predicted_index = probabilities.argmax()
predicted_label = labels[predicted_index] 
print('Predicted class:', predicted_label)

Predicted class: Samoyed
